Importing Files

In [ ]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import threading

Getting an access code to perform Data pulls

In [ ]:
# client_id = -------XXXXXX---------
# client_secret = -------YYYYY---------

client_id = '-------XXXXXX---------'
client_secret = '-------YYYYYY---------'

access_code = requests.post('https://id.twitch.tv/oauth2/token?client_id='+str(client_id)+'&client_secret='+str(client_secret)+'&grant_type=client_credentials')

access_token = json.loads(access_code.text)['access_token']

## The header is passes in each get request

headers = {
    'Authorization' : 'Bearer '+str(access_token),
    'Client-ID' : str(client_id)
}

Getting the top 100 games and the top 25 streams in each game.
Creating initial df to store subsequent pulls

In [ ]:
games_response = requests.get('https://api.twitch.tv/helix/games/top?first=100', headers=headers)

games_response_json = json.loads(games_response.text)
topgames_data = games_response_json['data']
topgames_df = pd.DataFrame.from_dict(json_normalize(topgames_data), orient='columns')

topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][0])+'&first=25', headers=headers)

topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)
topstreamsforgame_data = topstreamsforgame_response_json['data']
topstreamsforgame_df = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')

For loop to get top 25 streams for each game

In [ ]:
for i in range(1,19) :
    
        topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][i])+'&first=25', headers=headers)
    
        topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)
        topstreamsforgame_data = topstreamsforgame_response_json['data']
        topstreamsforgame_df_temp = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')
    
        frames = [topstreamsforgame_df, topstreamsforgame_df_temp]
        topstreamsforgame_df = pd.concat(frames, ignore_index=True)

Exporting the files to be used in Tableau Dashboard

In [ ]:
export_topgames_csv = topgames_df.to_csv (r'C:/Users/prparash/Documents/Python Scripts/topgames.csv', index = None, header=True)
export_topstreamsforgame_csv = topstreamsforgame_df.to_csv (r'C:/Users/prparash/Documents/Python Scripts/topstreams.csv', index = None, header=True)